In [1]:
sim_res = 1e9
awg_res = 5e8

# Quantum Volume

In [2]:
import tensorflow as tf
from qiskit.circuit.library import QuantumVolume as QuantumVolumeCircuit

2024-04-19 16:29:51.545256: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-19 16:29:51.546734: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 16:29:51.578430: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 16:29:51.578949: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 16:29:52.322736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## Create Lattice with qruise-toolset

In [3]:
from qruise.toolset.libraries.rydberg.lattice import SquareLattice
r12 = 4.0
lattice = SquareLattice(n_side=2, r_interatomic=r12)
lattice.draw()

In [4]:
from qruise.toolset.libraries.rydberg.rydbergsystem import RydbergSystem
hilbert_dim = 3
ryd_system = RydbergSystem(lattice=lattice, hilbert_dim=hilbert_dim)

In [5]:
rydberg_atom_list = ryd_system.rydberg_atom_list

Create the drives needed in the quantum volume sequence

$R_x$ and $R_y$ gates for all the qubits

In [6]:
import numpy as np
from qruise.toolset.objects import Quantity
phase_yp = Quantity(value=0.5 * np.pi, unit="rads")
phase_xp = Quantity(value=0.0, unit="rads")
phase_xm = Quantity(value=np.pi, unit="rads")
phase_ym = Quantity(value=-0.5 * np.pi, unit="rads")


In [7]:
qubit_names = [atom.name for atom in ryd_system.rydberg_atom_list]
# Global drives for sigle qubit gates
# Rx90p
ryd_system.add_drive(
    "amplitude-phase",
    "rx90p_global",
    channel="raman",
    connected=qubit_names,
    phase=phase_xp,
)
# Rx90m
ryd_system.add_drive(
    "amplitude-phase",
    "rx90m_global",
    channel="raman",
    connected=qubit_names,
    phase=phase_xm,
)
# Ry90p
ryd_system.add_drive(
    "amplitude-phase",
    "ry90p_global",
    channel="raman",
    connected=qubit_names,
    phase=phase_yp,
)
# Ry90m
ryd_system.add_drive(
    "amplitude-phase",
    "ry90m_global",
    channel="raman",
    connected=qubit_names,
    phase=phase_ym,
)

# Local drives for CZ gates
for atom in ryd_system.rydberg_atom_list:
    name = atom.name
    ryd_system.add_drive(
        "amplitude-phase",
        f"rydbergx{name}",
        channel="rydberg",
        connected=[name],
        phase=phase_xp,
    )


In [8]:
rabi_amplitude = 2e6 # MHz
t_pi_2 = np.around(1 / (4 * rabi_amplitude), 9)
t_pi = np.around(1 / (2 * rabi_amplitude), 9)

Create all possible instructions for the qubits

In [9]:
from qruise.toolset.signal import gates
from qruise.toolset.libraries.rydberg.envelopes import rect_envelope

target_labels = [int(atom.name) for atom in rydberg_atom_list]

gate = gates.Instruction(
    name="rx90p_global",
    t_end=t_pi_2,
    targets=target_labels,
    channels=["rx90p_global"],
    qiskit_name="rx90p",
)
gate.ideal = None
ryd_system.add_instruction(gate,
                           envelope=rect_envelope("rectangular_envelope", amp=2 * np.pi * rabi_amplitude*1e-6, t_final=t_pi_2),
                           drive_name="rx90p_global")
gate = gates.Instruction(
    name="rx90m_global",
    t_end=t_pi_2,
    targets=target_labels,
    channels=["rx90m_global"],
    qiskit_name="rx90m",
)
gate.ideal = None
ryd_system.add_instruction(gate,
                           envelope=rect_envelope("rectangular_envelope", amp= 2 * np.pi * rabi_amplitude*1e-6, t_final=t_pi_2),
                           drive_name="rx90m_global")

gate = gates.Instruction(
    name="ry90p_global",
    t_end=t_pi_2,
    targets=target_labels,
    channels=["ry90p_global"],
    qiskit_name="ry90p",
)
gate.ideal = None
ryd_system.add_instruction(gate,
                           envelope=rect_envelope("rectangular_envelope", amp=2 * np.pi * rabi_amplitude*1e-6, t_final=t_pi_2),
                           drive_name="ry90p_global")
gate = gates.Instruction(
    name="ry90m_global",
    t_end=t_pi_2,
    targets=target_labels,
    channels=["rx90m_global"],
    qiskit_name="ry90m",
)
gate.ideal = None
ryd_system.add_instruction(gate,
                           envelope=rect_envelope("rectangular_envelope", amp= 2 * np.pi * rabi_amplitude*1e-6, t_final=t_pi_2),
                           drive_name="ry90m_global")


# Create Rydberg state instructions for the CZ gates
for atom in rydberg_atom_list:
    ryd_pi = gates.Instruction(
        name=f"ryd_{atom.name}_pi",
        t_end=t_pi,
        channels=[f"rydbergx{atom.name}"],
        targets=[int(atom.name)],
    )
    ryd_pi.ideal = None
    ryd_system.add_instruction(instruction=ryd_pi, 
                               envelope=rect_envelope("rectangular_envelope", 
                                                      amp=2 * np.pi * rabi_amplitude*1e-6, 
                                                    t_final=t_pi), 
                                drive_name=f"rydbergx{atom.name}")
    ryd_2pi = gates.Instruction(
        name=f"ryd_{atom.name}_2pi",
        t_end=2 * t_pi,
        channels=[f"rydbergx{atom.name}"],
        targets=[int(atom.name)],
    )
    ryd_2pi.ideal = None
    ryd_system.add_instruction(instruction=ryd_2pi, 
                               envelope=rect_envelope("rectangular_envelope", amp=2 * np.pi * rabi_amplitude*1e-6, t_final=2 * t_pi), 
                               drive_name=f"rydbergx{atom.name}")

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: global. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: ryd. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: 0. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: pi. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: 2pi. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/signal/gates.py:211: UserWarning:

No ideal gate found for gate: 1. Use set_ideal() explicitly.

/home/marco/GitHub_Qruise/qruise-toolset/src/qru

In [10]:
qruise_instructions = ryd_system.instruction_list
qruise_instructions

[Instruction(rx90p_global[0,1,2,3]),
 Instruction(rx90m_global[0,1,2,3]),
 Instruction(ry90p_global[0,1,2,3]),
 Instruction(ry90m_global[0,1,2,3]),
 Instruction(ryd_0_pi[0]),
 Instruction(ryd_0_2pi[0]),
 Instruction(ryd_1_pi[1]),
 Instruction(ryd_1_2pi[1]),
 Instruction(ryd_2_pi[2]),
 Instruction(ryd_2_2pi[2]),
 Instruction(ryd_3_pi[3]),
 Instruction(ryd_3_2pi[3])]

Collect the model from the Rydberg system

In [11]:
ryd_system.create_model(max_excitations=5)

Set the generator

In [12]:
import qruise.toolset.generator.devices as devices
from qruise.toolset.generator.generator import Generator

ryd_system.sim_res = sim_res
ryd_system.awg_res = awg_res

awg_res = (
    ryd_system.awg_res
)  # number of samples per second; realistic limited resolution of an AWG
v2hz = 1e6
rise_time_value = 100e-9

# create a dict of all the devices
generator_devices = {
    "AWG": devices.AWG(name="awg", resolution=ryd_system.awg_res),
    "DigitalToAnalog": devices.DigitalToAnalog(name="dac"),
    "VoltsToHertz": devices.VoltsToHertz(
        name="v_to_hz", V_to_Hz=Quantity(value=v2hz, unit="Hz/V")
    ),
}

chains = {}
for atom in ryd_system.rydberg_atom_list:
    name = atom.name
    name_rydbergx = f"rydbergx{name}"
    chains[name_rydbergx] = {
        "AWG": [],
        "DigitalToAnalog": ["AWG"],
        "VoltsToHertz": ["DigitalToAnalog"],
    }
chains["rx90p_global"] = {
    "AWG": [],
    "DigitalToAnalog": ["AWG"],
    "VoltsToHertz": ["DigitalToAnalog"],
}
chains["rx90m_global"] = {
    "AWG": [],
    "DigitalToAnalog": ["AWG"],
    "VoltsToHertz": ["DigitalToAnalog"],
}
chains["ry90p_global"] = {
    "AWG": [],
    "DigitalToAnalog": ["AWG"],
    "VoltsToHertz": ["DigitalToAnalog"],
}
chains["ry90m_global"] = {
    "AWG": [],
    "DigitalToAnalog": ["AWG"],
    "VoltsToHertz": ["DigitalToAnalog"],
}
device_generator = Generator(
    devices=generator_devices,
    chains=chains,
)

ryd_system.set_generator(device_generator)

Create generator

In [13]:
ryd_system.create_experiment()

In [14]:
qruise_experiment = ryd_system.experiment

Qruise backend

In [15]:
from qruise.toolset.libraries.rydberg import RydbergAtom
from qruise.toolset.qiskit import QruiseProvider
qruise_provider = QruiseProvider()
qruise_backend = qruise_provider.get_backend("qruise_qasm_rydberg_simulator")
# Set qubit type to Rydberg atom
qruise_backend.qubit_type = RydbergAtom
qruise_backend.set_qruise_experiment(qruise_experiment)

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/experiment.py:153: UserWarning:

Using default simulation resolution in experiment, make sure this is what you expect (or you have set sim_res in the Instructions)



In [16]:
def state_transform_function() -> callable:
    def state_transform(
        state: tf.Tensor, gate_dict: dict, reshape_after: bool, reshape_before: bool
    ):
        # Gate dict name
        gate_name = gate_dict["name"]
        if gate_name == "cz":
            atom1 = gate_dict["qubits"][0]
            atom2 = gate_dict["qubits"][1]
            gate_names = [f"ryd_{atom1}_pi", f"ryd_{atom2}_2pi", f"ryd_{atom1}_pi"]
            targets = [[atom1], [atom2], [atom1]]
            for gate_index, gate_specs in enumerate(zip(gate_names, targets)):
                gate_name, gate_targets = gate_specs
                reshape_after = (
                None if reshape_after is None and gate_index == 3 else False
                )
                gate_dict["name"] = gate_name
                gate_dict["qubits"] = gate_targets
                state = qruise_experiment.lookup_propagator(
                    **gate_dict,
                    compute=False,
                ).propagate_state(
                    state, reshape_after=reshape_after, reshape_before=reshape_before
                )
                reshape_before = False
        elif "rx" in gate_name:
            gate_names = ["ry90p_global", "rz", "ry90m_global"]
            targets = [target_labels, gate_dict["qubits"], target_labels]
            for gate_index, gate_specs in enumerate(zip(gate_names, targets)):
                gate_name, gate_targets = gate_specs
                reshape_after = (
                None if reshape_after is None and gate_index == 3 else False
                )
                gate_dict["name"] = gate_name
                gate_dict["qubits"] = gate_targets
                state = qruise_experiment.lookup_propagator(
                    **gate_dict,
                    compute=False,
                ).propagate_state(
                    state, reshape_after=reshape_after, reshape_before=reshape_before
                )
                reshape_before = False
        elif "ry" in gate_name:
            gate_names = ["rx90m_global", "rz", "rx90p_global"]
            targets = [target_labels, gate_dict["qubits"], target_labels]
            for gate_index, gate_specs in enumerate(zip(gate_names, targets)):
                gate_name, gate_targets = gate_specs
                reshape_after = (
                None if reshape_after is None and gate_index == 3 else False
                )
                gate_dict["name"] = gate_name
                gate_dict["qubits"] = gate_targets
                state = qruise_experiment.lookup_propagator(
                    **gate_dict,
                    compute=False,
                ).propagate_state(
                    state, reshape_after=reshape_after, reshape_before=reshape_before
                )
                reshape_before = False
        else:
            raise ValueError("Not supported gate")
        return state

    return state_transform

In [17]:
qruise_backend._state_propagation_function = state_transform_function()

Run the Quantum Volume test

In [18]:
from utils import quantum_volume
quantum_volume(qruise_backend, ncircuits=150, nqubits=4, nshots=200)

/home/marco/GitHub_Qruise/qruise-toolset/src/qruise/toolset/qiskit/backends.py:332: UserWarning:

The following operations are not supported yet: ['measure', 'barrier']



Quantum Volume: 16
Percentage Heavy Outputs: 84.1%
Passed?: True



True